# Understanding Morphology

Here we will explain how BMTK and SONATA represent and understand the morphology of multi-compartment neuron, especially as it relates to the location of individual synapses. SONATA utilizes existing formats like NeuroML and SWC formats, so for most users they just need to download existing morphologies from databases like ModelDB or the Allen Cell-Types Database, and BMTK will recognize how to load and instantiate the downloaded files accordingly, without any extra thought from the user. But at times it may be necessary to understand what is going on at a deeper level.

When we are running simulations of thousands, or even millions, of morphologically realistic neurons trying to control and determine the location of each individual synapse would be impractical. By default BMTK allows users to write simple rules for each type of connection, such as
* Pyramidal cells have most of their synapses far away from the soma
* interneuron-to-interneuron connections may synapse at the soma, axon or basal dendrites

BMTK will then build the connections accordingly. But to help users better understand what is going on under the hood, we have put together this mini-tutorial.

## SWC Files

Suppose we have a cell with the morphology structure shown below:

![](../images/neuron_cartoon_legend.png)

The most common way to represent the morphology is through the **swc** format. An \*.swc file is a simple text file to represent the morphology using a space-delimited table. The morphology of all possible components of the neuron (including location of soma, axon, and all dendritic branches) is represented using a tree-like structure. Each line (except for the ones starting with a **#** which are used for comments) represents a point on the neuron. And each line also has a parent point (except for the root, often the soma, which has no parent).

So our neuron above may look like the following when converted to swc:

![](../images/swc_rep_text.png)


However what isn't captured by the swc is the location of synapses and gap junctions, receptors, and any dynamic part of the cell. Nor would we necessarily want it to be part of the swc. Cell-type morphology is likely to remain static (or alter only slightly) from simulation-to-simulation, but we may more frequently want to adjust the location and density of synapses.

The location of the synapses within an swc file are instead stored in the SONATA edges files using the reserved keywords **afferent_swc_id** and **afferent_swc_pos** (there are also reserved keywords for **efferent**/source-neuron synaptic position - but in most cases we don't use the **efferent** synaptic location since it usually comes from the axon).

* **afferent_swc_id** - An integer value that corresponds to the closest **id** in the swc file
* **afferent_swc_pos** - A floating-point number between (0, 1.0] used to indicate if the location is closer to the **id** (pos=1.0) or to the parent **pid** (pos=0.0).


We can open up the l4_l4_edges.h5 file to see more precisely how this is implemented:

![](../images/swc_id_hdf5_box.png)

We can see that source neuron with node-id \#72 is connected to neuron \#0 at-least 9 times (don't worry if your version of l4_l4_edges.h5 has different values, synapse positions are randomly generated). According to the "afferent_swc_id" for the first \#72 --> \#0 connection, the location of the synapse is at id 1691, half way (afferent_swc_pos=0.5) between swc point 1691 and its parent 1690.




## NEURON

When using ```BioNet```, a "biological" *model_type* with a corresponding swc *morphology* will be loaded into NEURON by bmtk automatically for you. The way NEURON represents the morphology of a given cell is a bit different than how it is stored in SWC format.

Because NEURON is more concerned about the flow of ions between compartments, and into and out of the membrane, the precise morphology and positions of any dendritic/axon branch is not required. In the stylized representation, when the morphology is loaded into NEURON it will divide the cell into distinct **sections**, which represents a non-bifurcating column of a particular diameter and length. Each section is assigned a unique id inside the NEURON code, as well as a more readable name. Each section may in turn contain one or more **segments**, which act as individual computational units.

As we can see the morphology described above might look like the following after it is parsed into NEURON:

![](../images/neuron_sections_w_segs.png)



As with SWC, the SONATA format has reserved keywords for keeping track of which sections individual synapses are located on, as well as where along the section are the synapses are located:

* **afferent_section_id** - An integer value corresponding to a NEURON assigned section id for the synapse location on the target cell.
* **afferent_section_pos** - A floating point value between (0.0, 1.0] that indicates where along the section the synapse is located. Note that this value only matters if a section has two or more segments, as within a segment the precise location is not relevant to calculating incoming current.

As before, these values are stored in our SONATA edges hdf5 files which we can see by opening the file with an HDF5 viewer.

## Abstracting away synapse location with BMTK

So how does one go about adding synaptic locations while keeping track of both the SWC and NEURON way of representing location within a cell's morphology? Do users have to manually assign the **section_id**, **section_pos**, **swc_id** and **swc_pos** for each and every synapse in a network? Luckily the BMTK ```NetworkBuilder``` has a few options for handling synapse placement (although one should feel free to specify every synapse if desired).



### Dynamic synaptic placement

The first option is to use special bmtk keywords **target_sections** and **distance_range** edge properties. When building the network, connecting to biophysical targets will look like the following:


```python
  net.add_edges(
      source={'type': 'basket'},
      target={'type': 'pyramidal', 'model_type': 'biophysical'},
      connection_rule=16,
      target_sections=['basal', 'apical', 'axon'],
      distance_range=[30.0, 100.0],
      ....
  )
```

* **connection_rule**: An integer, function, or array containing the number of connections between two cells 
* **target_sections**: A list of cell regions (soma, basal, apical, axon, other) that will receive synapses
* **distance_range**: A closed range, along any of the specific **target_sections**, in microns, where synapses will be allowed to be located. This value is defined as the arc-length distance from the center of the soma.

Using our neuron cartoon, we can see there are six branches/sections that match our criteria; all highlighted in yellow. 

![](../images/neuron_cartoon_targets.png)

When ```BioNet``` runs the simulation, it will randomly place synapses within the highlighted regions when building these types of "basket" --> "pyramidal" connections (there is no guarantee on the distribution, so it's possible that all synapses will be put in the same basal dendrite branch).

If you want to use all of one region you can make the end range infinite or a really high number. In the below example we want to target all apical dendritic sections as candidate locations to place synapses - but we don't know how far the apical branches extend or even where it begins (it may branch off a basal dendrite). 

```python
  net.add_edges(
      source={'type': 'pyramidal'},
      target={'type': 'pyramidal', 'model_type': 'biophysical'},
      connection_rule=my_connection_rule,
      target_sections=['apical'],
      distance_range=[0.0, 1.0e20],
      ....
  )
```

![](../images/neuron_cartoon_target_apical.png)


However, one problem with this method is that individual synapse locations are not stored on the edges files. Which means if we want visualize them, analyze their density patterns, or adjust individual synapses we cannot do so.

### Explicit saving of synapse locations

The bmtk ```NetworkBuilder``` includes built-in functions like ```rand_syn_location``` and others that we can use to explicitly find **swc** and NEURON **section** ID's and positions. The function can take parameters like *target_sections* and *distance_range* and actually calculate the required values:

```python
from bmtk.builder.bionet import rand_syn_locations


sec_ids, sec_pos, swc_ids, swc_pos = rand_syn_locations(
    N=16,
    sections=['basal', 'apical', 'axon'],
    distance_ranges=[30.0, 100.0],
    morphology='compoents/morphologies/Pyramidal_156892.swc'
)
    
```

It will return lists of 16 (each index coming from the same value) **afferent_section_ids**, **afferent_section_positions**, **afferent_swc_ids** and **afferent_swc_pos**"

To use with the ```NetworkBuilder```:


```python
conns = net.add_edges(
    source={'type': 'basket'},
    target={'type': 'pyramidal', 'model_type': 'biophysical'},
    connection_rule=16,
    syn_weight=0.00025,
    delay=2.0,
    dynamics_params='GABA_InhToExc.json',
    model_template='Exp2Syn',
)
conns.add_properties(
    ['afferent_section_id', 'afferent_section_pos', 'afferent_swc_id', 'afferent_swc_pos'],
    rule=rand_syn_locations,
    rule_params={
        'sections': ['basal', 'apical', 'axon'], 
        'distance_range': [30.0, 100.0],
        'morphology_dir': 'components/morphologies' 
    },
    dtypes=[np.int, np.float, np.int, np.float]
)
```